# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khatanga,71.9667,102.5000,18.50,94,43,8.84,RU,1682606089
1,1,vanersborg,58.3807,12.3234,51.91,34,0,17.27,SE,1682606090
2,2,banda aceh,5.5577,95.3222,81.55,79,100,5.08,ID,1682606090
3,3,antsohihy,-14.8796,47.9875,83.14,65,46,5.44,MG,1682606090
4,4,victoria,22.2855,114.1577,73.96,89,86,13.00,HK,1682606029


In [22]:
# Capitalize all City Names
city_data_df.City = city_data_df.City.str.title()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Khatanga,71.9667,102.5000,18.50,94,43,8.84,RU,1682606089
1,1,Vanersborg,58.3807,12.3234,51.91,34,0,17.27,SE,1682606090
2,2,Banda Aceh,5.5577,95.3222,81.55,79,100,5.08,ID,1682606090
3,3,Antsohihy,-14.8796,47.9875,83.14,65,46,5.44,MG,1682606090
4,4,Victoria,22.2855,114.1577,73.96,89,86,13.00,HK,1682606029


In [23]:
# Rename Lat and Lng for better presentation in Map
city_data_df = city_data_df.rename(columns={'Lat' : 'Latitude', 'Lng' : 'Longitude'})
city_data_df.columns

Index(['City_ID', 'City', 'Latitude', 'Longitude', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date'],
      dtype='object')

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    size = "Humidity",
    color = "City"
)

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

### Criteria:
### 1. A max temperature lower than 85 degrees but higher than 65

### 2. Wind speed less than 10 m/s

### 3. Cloudiness less than 5

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 85) & (city_data_df['Max Temp'] > 65) & (city_data_df['Wind Speed'] < 5) & (city_data_df['Cloudiness'] < 10)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,Razole,16.4833,81.8333,80.78,82,8,3.11,IN,1682606097
122,122,Pisco,-13.7000,-76.2167,74.01,78,7,3.44,PE,1682606114
150,150,Las Vegas,36.1750,-115.1372,70.43,21,0,4.61,US,1682605918
204,204,Formosa,-26.1775,-58.1781,75.96,65,0,4.61,AR,1682605866
219,219,Talara,-4.5772,-81.2719,81.36,78,0,3.44,PE,1682606134
309,309,Miandrivazo,-19.5167,45.4667,83.52,53,0,4.12,MG,1682606149
314,314,Vangaindrano,-23.3500,47.6000,70.97,85,8,4.99,MG,1682606150
315,315,Laguna,38.4210,-121.4238,69.08,75,0,3.44,US,1682606150
396,396,Saiha,22.4833,92.9667,72.64,51,3,1.39,IN,1682606164
414,414,Dixon,38.4455,-121.8233,66.22,73,0,0.00,US,1682606168


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

df1 = ideal_weather_df.copy()
hotel_df = df1[['City', 'Country', 'Latitude', 'Longitude', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity,Hotel Name
46,Razole,IN,16.4833,81.8333,82,
122,Pisco,PE,-13.7000,-76.2167,78,
150,Las Vegas,US,36.1750,-115.1372,21,
204,Formosa,AR,-26.1775,-58.1781,65,
219,Talara,PE,-4.5772,-81.2719,78,
309,Miandrivazo,MG,-19.5167,45.4667,53,
314,Vangaindrano,MG,-23.3500,47.6000,85,
315,Laguna,US,38.4210,-121.4238,75,
396,Saiha,IN,22.4833,92.9667,51,
414,Dixon,US,38.4455,-121.8233,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
category = "accommodation.hotel"
limit = 20
radius = 10000
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"

params = {
    "categories":category,
    #"filters":filters,
    "limit":limit,
    #"bias":bias,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Latitude']
    longitude = hotel_df.loc[index, 'Longitude']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Razole - nearest hotel: No hotel found
Pisco - nearest hotel: La Portada
Las Vegas - nearest hotel: Downtown Grand
Formosa - nearest hotel: Hotel Hostal del Rey
Talara - nearest hotel: Hospedaje El Aypate
Miandrivazo - nearest hotel: Baobab
Vangaindrano - nearest hotel: Fanilo
Laguna - nearest hotel: Holiday Inn Express & Suites
Saiha - nearest hotel: No hotel found
Dixon - nearest hotel: Best Western Plus Dixon Davis
Buala - nearest hotel: No hotel found
Benjamin Aceval - nearest hotel: No hotel found
Rodeo - nearest hotel: Days Inn by Wyndham Pinole Berkeley
Puerto Escondido - nearest hotel: Hotel Barlovento
Mareeba - nearest hotel: The Gateway Hotel
Trindade - nearest hotel: Hotel Liguori
Sola - nearest hotel: No hotel found
Honiara - nearest hotel: King Solomon Hotel


,City,Country,Latitude,Longitude,Humidity,Hotel Name
46,Razole,IN,16.4833,81.8333,82,No hotel found
122,Pisco,PE,-13.7000,-76.2167,78,La Portada
150,Las Vegas,US,36.1750,-115.1372,21,Downtown Grand
204,Formosa,AR,-26.1775,-58.1781,65,Hotel Hostal del Rey
219,Talara,PE,-4.5772,-81.2719,78,Hospedaje El Aypate
309,Miandrivazo,MG,-19.5167,45.4667,53,Baobab
314,Vangaindrano,MG,-23.3500,47.6000,85,Fanilo
315,Laguna,US,38.4210,-121.4238,75,Holiday Inn Express & Suites
396,Saiha,IN,22.4833,92.9667,51,No hotel found
414,Dixon,US,38.4455,-121.8233,73,Best Western Plus Dixon Davis


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# Configure the map plot

# Configure the map plot
ideal_locations_map = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    size = "Humidity",
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
ideal_locations_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)